In [1]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np

In [2]:
nba_teams = teams.get_teams()
df_teams = pd.DataFrame(nba_teams)
df_teams.head(60)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,San Francisco,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


# Selección de partido
Se va a seleccionar el partido jugado el 13/12/2025 entre `okc` y `spurs`.

In [3]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguegamelog

In [4]:
partidos_temporada = leaguegamefinder.LeagueGameFinder(season_nullable="2025-26")
df_partidos_temporada = partidos_temporada.get_data_frames()[0]
df_partido_okc = df_partidos_temporada[((df_partidos_temporada['GAME_DATE'] == "2025-12-13") & 
                                    (df_partidos_temporada['TEAM_ABBREVIATION'] == 'OKC'))]
df_partido_okc

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
101,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,0.889,14,37,51,20,13,4,14,25,-2.0


In [5]:
game_id = df_partido_okc['GAME_ID'].iloc[0]
print(game_id)

0022501230


In [6]:
def extraer_sim(matchup):
    resultado = matchup[4]
    return resultado

def extraer_primero(matchup):
    resultado = matchup[0:3]
    return resultado

In [7]:
df_partidos_temporada['matchup_sim'] = df_partidos_temporada['MATCHUP'].loc[:].apply(extraer_sim)
df_partidos_temporada['matchup_pri'] = df_partidos_temporada['MATCHUP'].loc[:].apply(extraer_primero)
df_partidos_temporada['HA'] = np.where((df_partidos_temporada['TEAM_ABBREVIATION'] == df_partidos_temporada['matchup_pri'])&
    (df_partidos_temporada['matchup_sim'] == '@'), 'AWAY', np.where(
        (df_partidos_temporada['TEAM_ABBREVIATION'] != df_partidos_temporada['matchup_pri']) &
        (df_partidos_temporada['matchup_sim'] == 'v'),'AWAY', 'HOME'
    )
)
df_partidos_temporada.drop(columns=['matchup_sim', 'matchup_pri'], inplace=True)
df_partidos_temporada

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA
0,22025,1610612761,TOR,Toronto Raptors,0022500397,2025-12-21,TOR @ BKN,L,240,81,...,9,30,39,23,6,5,15,19,-15.0,AWAY
1,22025,1610612750,MIN,Minnesota Timberwolves,0022500400,2025-12-21,MIN vs. MIL,W,240,103,...,11,35,46,28,10,4,10,18,3.0,HOME
2,22025,1610612764,WAS,Washington Wizards,0022500399,2025-12-21,WAS vs. SAS,L,240,113,...,8,35,43,20,5,7,5,25,-11.0,HOME
3,22025,1610612737,ATL,Atlanta Hawks,0022500396,2025-12-21,ATL vs. CHI,L,239,150,...,6,29,35,39,6,7,10,25,-2.0,HOME
4,22025,1610612748,MIA,Miami Heat,0022500398,2025-12-21,MIA @ NYK,L,240,125,...,12,33,45,26,5,1,11,14,-7.0,AWAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,12025,15016,MEL,Melbourne United,0012500009,2025-10-03,MEL @ NOP,L,240,97,...,13,35,48,23,7,1,17,18,-10.2,AWAY
987,12025,15016,MEL,Melbourne United,0012500009,2025-10-03,MEL @ NOP,None,1,0,...,0,0,0,0,0,0,0,0,0.0,AWAY
988,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,10,40,50,29,9,5,16,31,22.0,AWAY
989,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,20,38,58,19,13,2,17,26,15.0,HOME


In [8]:
df_partidos_temporada['GAME_ID'] = df_partidos_temporada['GAME_ID'].astype(str)
df_partidos_temporada['TEAM_ID'] = df_partidos_temporada['TEAM_ID'].astype(str)
df_partidos_temporada['GAME_TEAM_ID'] = df_partidos_temporada['GAME_ID']+'-'+df_partidos_temporada['TEAM_ID']
df_partidos_temporada = df_partidos_temporada.drop_duplicates(subset=['GAME_TEAM_ID'])
df_partidos_temporada

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA,GAME_TEAM_ID
0,22025,1610612761,TOR,Toronto Raptors,0022500397,2025-12-21,TOR @ BKN,L,240,81,...,30,39,23,6,5,15,19,-15.0,AWAY,0022500397-1610612761
1,22025,1610612750,MIN,Minnesota Timberwolves,0022500400,2025-12-21,MIN vs. MIL,W,240,103,...,35,46,28,10,4,10,18,3.0,HOME,0022500400-1610612750
2,22025,1610612764,WAS,Washington Wizards,0022500399,2025-12-21,WAS vs. SAS,L,240,113,...,35,43,20,5,7,5,25,-11.0,HOME,0022500399-1610612764
3,22025,1610612737,ATL,Atlanta Hawks,0022500396,2025-12-21,ATL vs. CHI,L,239,150,...,29,35,39,6,7,10,25,-2.0,HOME,0022500396-1610612737
4,22025,1610612748,MIA,Miami Heat,0022500398,2025-12-21,MIA @ NYK,L,240,125,...,33,45,26,5,1,11,14,-7.0,AWAY,0022500398-1610612748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,12025,1610612740,NOP,New Orleans Pelicans,0012500009,2025-10-03,NOP vs. MEL,W,240,107,...,33,46,20,11,8,16,22,10.0,HOME,0012500009-1610612740
986,12025,15016,MEL,Melbourne United,0012500009,2025-10-03,MEL @ NOP,L,240,97,...,35,48,23,7,1,17,18,-10.2,AWAY,0012500009-15016
988,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,40,50,29,9,5,16,31,22.0,AWAY,0012500001-1610612756
989,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,38,58,19,13,2,17,26,15.0,HOME,0012500008-1610612752


In [9]:
equipos = list(df_teams['id'].astype(str))
equipos

['1610612737',
 '1610612738',
 '1610612739',
 '1610612740',
 '1610612741',
 '1610612742',
 '1610612743',
 '1610612744',
 '1610612745',
 '1610612746',
 '1610612747',
 '1610612748',
 '1610612749',
 '1610612750',
 '1610612751',
 '1610612752',
 '1610612753',
 '1610612754',
 '1610612755',
 '1610612756',
 '1610612757',
 '1610612758',
 '1610612759',
 '1610612760',
 '1610612761',
 '1610612762',
 '1610612763',
 '1610612764',
 '1610612765',
 '1610612766']

In [10]:
df_partidos_temporada = df_partidos_temporada.loc[
    df_partidos_temporada["TEAM_ID"].isin(equipos)
]
df_partidos_temporada

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA,GAME_TEAM_ID
0,22025,1610612761,TOR,Toronto Raptors,0022500397,2025-12-21,TOR @ BKN,L,240,81,...,30,39,23,6,5,15,19,-15.0,AWAY,0022500397-1610612761
1,22025,1610612750,MIN,Minnesota Timberwolves,0022500400,2025-12-21,MIN vs. MIL,W,240,103,...,35,46,28,10,4,10,18,3.0,HOME,0022500400-1610612750
2,22025,1610612764,WAS,Washington Wizards,0022500399,2025-12-21,WAS vs. SAS,L,240,113,...,35,43,20,5,7,5,25,-11.0,HOME,0022500399-1610612764
3,22025,1610612737,ATL,Atlanta Hawks,0022500396,2025-12-21,ATL vs. CHI,L,239,150,...,29,35,39,6,7,10,25,-2.0,HOME,0022500396-1610612737
4,22025,1610612748,MIA,Miami Heat,0022500398,2025-12-21,MIA @ NYK,L,240,125,...,33,45,26,5,1,11,14,-7.0,AWAY,0022500398-1610612748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,12025,1610612747,LAL,Los Angeles Lakers,0012500001,2025-10-03,LAL vs. PHX,L,240,81,...,35,46,10,9,7,22,25,-22.0,HOME,0012500001-1610612747
985,12025,1610612740,NOP,New Orleans Pelicans,0012500009,2025-10-03,NOP vs. MEL,W,240,107,...,33,46,20,11,8,16,22,10.0,HOME,0012500009-1610612740
988,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,40,50,29,9,5,16,31,22.0,AWAY,0012500001-1610612756
989,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,38,58,19,13,2,17,26,15.0,HOME,0012500008-1610612752


In [11]:
df_partidos_temporada.duplicated(subset=['GAME_TEAM_ID']).sum()

np.int64(0)

In [12]:
df_partidos_temporada[df_partidos_temporada.duplicated(subset=['GAME_TEAM_ID'], keep=False)]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA,GAME_TEAM_ID


In [13]:
partido = df_partidos_temporada[df_partidos_temporada['GAME_ID'] == game_id]
partido

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA,GAME_TEAM_ID
99,22025,1610612759,SAS,San Antonio Spurs,0022501230,2025-12-13,SAS @ OKC,W,241,111,...,37,52,21,9,6,19,23,2.0,AWAY,0022501230-1610612759
101,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,37,51,20,13,4,14,25,-2.0,HOME,0022501230-1610612760


In [14]:
from nba_api.stats.static import players

In [15]:
from nba_api.stats.static import players
import pandas as pd
import pandas as pd

pd.set_option("display.max_colwidth", None)


nba_players = players.get_players()
df_players = pd.DataFrame(nba_players)

df_players["headshot"] = df_players["id"].apply(
    lambda player_id: f"https://cdn.nba.com/headshots/nba/latest/1040x760/{int(player_id)}.png"
)

df_players.head(10)

,id,full_name,first_name,last_name,is_active,headshot
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False,https://cdn.nba.com/headshots/nba/latest/1040x760/76001.png
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False,https://cdn.nba.com/headshots/nba/latest/1040x760/76002.png
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False,https://cdn.nba.com/headshots/nba/latest/1040x760/76003.png
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False,https://cdn.nba.com/headshots/nba/latest/1040x760/51.png
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False,https://cdn.nba.com/headshots/nba/latest/1040x760/1505.png
5,949,Shareef Abdur-Rahim,Shareef,Abdur-Rahim,False,https://cdn.nba.com/headshots/nba/latest/1040x760/949.png
6,76005,Tom Abernethy,Tom,Abernethy,False,https://cdn.nba.com/headshots/nba/latest/1040x760/76005.png
7,76006,Forest Able,Forest,Able,False,https://cdn.nba.com/headshots/nba/latest/1040x760/76006.png
8,76007,John Abramovic,John,Abramovic,False,https://cdn.nba.com/headshots/nba/latest/1040x760/76007.png
9,203518,Alex Abrines,Alex,Abrines,False,https://cdn.nba.com/headshots/nba/latest/1040x760/203518.png


In [16]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import boxscoretraditionalv3
import pandas as pd

In [17]:
tiros = shotchartdetail.ShotChartDetail(
    team_id= 0,
    player_id= 0,
    season_type_all_star= 'Regular Season',
    season_nullable= '2025-26'
)

df_tiros =  tiros.get_data_frames()[0]
df_tiros.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022500001,10,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,11,26,...,Center(C),Less Than 8 ft.,1,0,13,1,1,20251021,OKC,HOU
1,Shot Chart Detail,0022500001,17,1631095,Jabari Smith Jr.,1610612745,Houston Rockets,1,11,2,...,Center(C),Less Than 8 ft.,0,-7,-5,1,1,20251021,OKC,HOU
2,Shot Chart Detail,0022500001,18,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,10,50,...,Center(C),Less Than 8 ft.,7,64,39,1,1,20251021,OKC,HOU
3,Shot Chart Detail,0022500001,27,201142,Kevin Durant,1610612745,Houston Rockets,1,10,7,...,Center(C),Less Than 8 ft.,0,0,-1,1,1,20251021,OKC,HOU
4,Shot Chart Detail,0022500001,29,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,9,52,...,Center(C),8-16 ft.,13,59,116,1,1,20251021,OKC,HOU


In [18]:
df_tiros['GAME_GEVENT_ID'] = df_tiros['GAME_ID'].astype(str)+'-'+df_tiros['GAME_EVENT_ID'].astype(str)
df_tiros['GAME_TEAM_ID'] = df_tiros['GAME_ID'].astype(str)+'-'+df_tiros['TEAM_ID'].astype(str)
df_tiros.duplicated(subset=['GAME_GEVENT_ID']).sum()

np.int64(0)

In [19]:
df_tiros.isnull().sum()

GRID_TYPE              0
GAME_ID                0
GAME_EVENT_ID          0
PLAYER_ID              0
PLAYER_NAME            0
TEAM_ID                0
TEAM_NAME              0
PERIOD                 0
MINUTES_REMAINING      0
SECONDS_REMAINING      0
EVENT_TYPE             0
ACTION_TYPE            0
SHOT_TYPE              0
SHOT_ZONE_BASIC        0
SHOT_ZONE_AREA         0
SHOT_ZONE_RANGE        0
SHOT_DISTANCE          0
LOC_X                  0
LOC_Y                  0
SHOT_ATTEMPTED_FLAG    0
SHOT_MADE_FLAG         0
GAME_DATE              0
HTM                    0
VTM                    0
GAME_GEVENT_ID         0
GAME_TEAM_ID           0
dtype: int64

In [20]:
df_tiros.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,GAME_GEVENT_ID,GAME_TEAM_ID
0,Shot Chart Detail,0022500001,10,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,11,26,...,1,0,13,1,1,20251021,OKC,HOU,0022500001-10,0022500001-1610612760
1,Shot Chart Detail,0022500001,17,1631095,Jabari Smith Jr.,1610612745,Houston Rockets,1,11,2,...,0,-7,-5,1,1,20251021,OKC,HOU,0022500001-17,0022500001-1610612745
2,Shot Chart Detail,0022500001,18,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,10,50,...,7,64,39,1,1,20251021,OKC,HOU,0022500001-18,0022500001-1610612760
3,Shot Chart Detail,0022500001,27,201142,Kevin Durant,1610612745,Houston Rockets,1,10,7,...,0,0,-1,1,1,20251021,OKC,HOU,0022500001-27,0022500001-1610612745
4,Shot Chart Detail,0022500001,29,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,9,52,...,13,59,116,1,1,20251021,OKC,HOU,0022500001-29,0022500001-1610612760


In [21]:
df_tiros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35373 entries, 0 to 35372
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   GRID_TYPE            35373 non-null  object
 1   GAME_ID              35373 non-null  object
 2   GAME_EVENT_ID        35373 non-null  int64 
 3   PLAYER_ID            35373 non-null  int64 
 4   PLAYER_NAME          35373 non-null  object
 5   TEAM_ID              35373 non-null  int64 
 6   TEAM_NAME            35373 non-null  object
 7   PERIOD               35373 non-null  int64 
 8   MINUTES_REMAINING    35373 non-null  int64 
 9   SECONDS_REMAINING    35373 non-null  int64 
 10  EVENT_TYPE           35373 non-null  object
 11  ACTION_TYPE          35373 non-null  object
 12  SHOT_TYPE            35373 non-null  object
 13  SHOT_ZONE_BASIC      35373 non-null  object
 14  SHOT_ZONE_AREA       35373 non-null  object
 15  SHOT_ZONE_RANGE      35373 non-null  object
 16  SHOT

In [30]:
boxscore = boxscoretraditionalv3.BoxScoreTraditionalV3(
    game_id= game_id
)
df_boxscore = boxscore.get_data_frames()[0]
df_boxscore.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1631114,Jalen,Williams,J. Williams,...,1,6,7,4,4,0,2,2,17,-18.0
1,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1631096,Chet,Holmgren,C. Holmgren,...,0,7,7,0,0,1,1,3,17,18.0
2,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1628392,Isaiah,Hartenstein,I. Hartenstein,...,8,1,9,1,1,1,3,3,10,-7.0
3,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,0,3,3,2,0,1,0,3,3,-10.0
4,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1628983,Shai,Gilgeous-Alexander,S. Gilgeous-Alexander,...,0,4,4,5,0,1,5,4,29,2.0


In [34]:
df_boxscore['game_player_id'] = df_boxscore['gameId'].astype(str)+'-'+df_boxscore['personId'].astype(str)
df_boxscore['game_team_id'] = df_boxscore['gameId'].astype(str)+'-'+df_boxscore['teamId'].astype(str)
df_boxscore.duplicated(subset=['game_player_id']).sum()


np.int64(0)

In [35]:
df_boxscore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gameId                   28 non-null     object 
 1   teamId                   28 non-null     int64  
 2   teamCity                 28 non-null     object 
 3   teamName                 28 non-null     object 
 4   teamTricode              28 non-null     object 
 5   teamSlug                 28 non-null     object 
 6   personId                 28 non-null     int64  
 7   firstName                28 non-null     object 
 8   familyName               28 non-null     object 
 9   nameI                    28 non-null     object 
 10  playerSlug               28 non-null     object 
 11  position                 28 non-null     object 
 12  comment                  28 non-null     object 
 13  jerseyNum                28 non-null     object 
 14  minutes                  28 

In [23]:
from nba_api.stats.static import teams
import pandas as pd

# Obtener equipos NBA
nba_teams = teams.get_teams()
df_teams = pd.DataFrame(nba_teams)

# Crear columna logo a partir del team id
df_teams["logo"] = df_teams["id"].apply(
    lambda team_id: f"https://cdn.nba.com/logos/nba/{int(team_id)}/primary/L/logo.svg"
)

# Opcional: mostrar URLs completas
pd.set_option("display.max_colwidth", None)

# Ver resultado
df_teams.head(10)


,id,full_name,abbreviation,nickname,city,state,year_founded,logo
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,https://cdn.nba.com/logos/nba/1610612737/primary/L/logo.svg
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,https://cdn.nba.com/logos/nba/1610612738/primary/L/logo.svg
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,https://cdn.nba.com/logos/nba/1610612739/primary/L/logo.svg
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,https://cdn.nba.com/logos/nba/1610612740/primary/L/logo.svg
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,https://cdn.nba.com/logos/nba/1610612741/primary/L/logo.svg
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980,https://cdn.nba.com/logos/nba/1610612742/primary/L/logo.svg
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976,https://cdn.nba.com/logos/nba/1610612743/primary/L/logo.svg
7,1610612744,Golden State Warriors,GSW,Warriors,San Francisco,California,1946,https://cdn.nba.com/logos/nba/1610612744/primary/L/logo.svg
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967,https://cdn.nba.com/logos/nba/1610612745/primary/L/logo.svg
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970,https://cdn.nba.com/logos/nba/1610612746/primary/L/logo.svg


In [1]:
import sys
from pathlib import Path

ROOT = Path().resolve().parent  # sube un nivel desde notebooks
sys.path.append(str(ROOT))

In [2]:
from src.etl.extract.games import fetch_games

In [3]:
df_prueba = fetch_games(['2024-25', '2025-26'])

In [6]:
df_prueba.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,SEASON_NUM,HA,GAME_TEAM_ID
0,22024,1610612747,LAL,Los Angeles Lakers,0022401199,2025-04-13,LAL @ POR,L,239,81,...,42,21,9,11,20,21,-28.0,2024-25,AWAY,0022401199-1610612747
1,22024,1610612749,MIL,Milwaukee Bucks,0022401192,2025-04-13,MIL vs. DET,W,265,140,...,38,30,5,7,20,17,7.0,2024-25,HOME,0022401192-1610612749
2,22024,1610612752,NYK,New York Knicks,0022401188,2025-04-13,NYK @ BKN,W,240,113,...,40,27,10,3,11,25,8.0,2024-25,AWAY,0022401188-1610612752
3,22024,1610612761,TOR,Toronto Raptors,0022401197,2025-04-13,TOR @ SAS,L,240,118,...,42,32,8,1,13,23,-7.0,2024-25,AWAY,0022401197-1610612761
4,22024,1610612765,DET,Detroit Pistons,0022401192,2025-04-13,DET @ MIL,L,264,133,...,42,35,9,3,13,19,-7.0,2024-25,AWAY,0022401192-1610612765


In [8]:
from src.etl.extract.boxscore import fetch_game_ids_from_db
seasons = ['2024-25','2025-26']
game_id = fetch_game_ids_from_db(seasons)
game_id[0:10]

['0022400799',
 '0022500125',
 '0022400804',
 '0022400966',
 '0022400547',
 '0022500350',
 '0022500266',
 '0022401155',
 '0022400764',
 '0022400786']

In [21]:
from nba_api.stats.endpoints import boxscoretraditionalv3
from nba_api.stats.endpoints import boxscoretraditionalv2
import pandas as pd

In [14]:
dfs = []
game_id_intentado = []
for gid in game_id:
    game_id_intentado.append(gid)
    box = boxscoretraditionalv3.BoxScoreTraditionalV3(game_id=gid, timeout=120)
    df_players = box.get_data_frames()[0]
    dfs.append(df_players)

df_concat = pd.concat(dfs, axis=0)

df_concat.head()

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=120)

In [17]:
game_id_intentado

['0022400799',
 '0022500125',
 '0022400804',
 '0022400966',
 '0022400547',
 '0022500350',
 '0022500266',
 '0022401155',
 '0022400764',
 '0022400786',
 '0022400179',
 '0022400193',
 '0022500212',
 '0022400584',
 '0022401098',
 '0022400184',
 '0022400323',
 '0022400893',
 '0022500153',
 '0022400701',
 '0022400064',
 '0022400439',
 '0022500316',
 '0022400046',
 '0022400874',
 '0022400817',
 '0022400894',
 '0022400911',
 '0022400690']

In [40]:
box1 = boxscoretraditionalv3.BoxScoreTraditionalV3(game_id='0022401194')
df_players1 = box1.get_data_frames()[0]
df_players1

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1630205,Lamar,Stevens,L. Stevens,...,1,5,6,3,0,0,3,3,31,34.0
1,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1641713,GG,Jackson,G. Jackson,...,1,3,4,3,0,0,2,2,5,15.0
2,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1628963,Marvin,Bagley III,M. Bagley III,...,5,7,12,3,1,0,1,2,25,24.0
3,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1631246,Vince,Williams Jr.,V. Williams Jr.,...,5,6,11,5,0,1,0,1,12,12.0
4,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1642285,Cam,Spencer,C. Spencer,...,2,0,2,7,3,0,0,3,23,18.0
5,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1630643,Jay,Huff,J. Huff,...,3,3,6,2,2,6,3,3,22,16.0
6,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1642530,Yuki,Kawamura,Y. Kawamura,...,0,5,5,5,1,0,1,1,12,29.0
7,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1629723,John,Konchar,J. Konchar,...,2,5,7,3,6,0,1,0,2,27.0
8,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1630583,Santi,Aldama,S. Aldama,...,0,0,0,0,0,0,0,0,0,0.0
9,0022401194,1610612763,Memphis,Grizzlies,MEM,grizzlies,1641744,Zach,Edey,Z. Edey,...,0,0,0,0,0,0,0,0,0,0.0


In [36]:
box2 = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id='0022400912')
df_players2 = box1.get_data_frames()[0]
df_players2

/tmp/ipykernel_27776/4041847956.py:1: DeprecationWarning: BoxScoreTraditionalV2 is deprecated and will be removed in a future version. Please use BoxScoreTraditionalV3 instead. Data is no longer being published for BoxScoreTraditionalV2 as of the 2025-26 NBA season.
  box2 = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id='0022400912')


,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1630182,Josh,Green,J. Green,...,2,2,4,5,0,0,1,3,0,-2.0
1,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1628970,Miles,Bridges,M. Bridges,...,3,9,12,5,1,1,2,1,26,2.0
2,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1631217,Moussa,Diabaté,M. Diabaté,...,10,5,15,2,1,0,2,2,16,-4.0
3,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1641733,Nick,Smith Jr.,N. Smith Jr.,...,0,2,2,2,0,0,0,1,19,4.0
4,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1641878,Damion,Baugh,D. Baugh,...,1,1,2,4,0,0,2,0,14,3.0
5,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1642275,Tidjane,Salaün,T. Salaün,...,1,2,3,2,1,0,1,0,9,3.0
6,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1630201,Malachi,Flynn,M. Flynn,...,2,3,5,1,1,0,1,3,9,0.0
7,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1629610,DaQuan,Jeffries,D. Jeffries,...,0,0,0,1,0,0,0,0,3,-2.0
8,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,201959,Taj,Gibson,T. Gibson,...,1,1,2,1,0,0,1,3,4,7.0
9,0022400912,1610612766,Charlotte,Hornets,CHA,hornets,1631111,Wendell,Moore Jr.,W. Moore Jr.,...,2,3,5,2,0,1,0,0,5,4.0


In [28]:
df_players2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gameId                   24 non-null     object 
 1   teamId                   24 non-null     int64  
 2   teamCity                 24 non-null     object 
 3   teamName                 24 non-null     object 
 4   teamTricode              24 non-null     object 
 5   teamSlug                 24 non-null     object 
 6   personId                 24 non-null     int64  
 7   firstName                24 non-null     object 
 8   familyName               24 non-null     object 
 9   nameI                    24 non-null     object 
 10  playerSlug               24 non-null     object 
 11  position                 24 non-null     object 
 12  comment                  24 non-null     object 
 13  jerseyNum                24 non-null     object 
 14  minutes                  24 

In [29]:
df_players1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gameId                   24 non-null     object 
 1   teamId                   24 non-null     int64  
 2   teamCity                 24 non-null     object 
 3   teamName                 24 non-null     object 
 4   teamTricode              24 non-null     object 
 5   teamSlug                 24 non-null     object 
 6   personId                 24 non-null     int64  
 7   firstName                24 non-null     object 
 8   familyName               24 non-null     object 
 9   nameI                    24 non-null     object 
 10  playerSlug               24 non-null     object 
 11  position                 24 non-null     object 
 12  comment                  24 non-null     object 
 13  jerseyNum                24 non-null     object 
 14  minutes                  24 